This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true(train_quad, test_quad, outdir, label):
    """
    Method to create a parity plot (predicted vs. true values)

    Args:

        train_quad: (tuple), tuple containing 4 numpy arrays: true training y data, predicted training y data,

        training metric data, and groups used in training

        test_quad: (tuple), tuple containing 4 numpy arrays: true test y data, predicted test y data,

        testing metric data, and groups used in testing

        outdir: (str), path to save plots to

        label: (str), label used for axis labeling

    Returns:

        None

    """
    filenames = list()
    y_train_true, y_train_pred, train_metrics, train_groups = train_quad
    y_test_true, y_test_pred, test_metrics, test_groups = test_quad

    # make diagonal line from absolute min to absolute max of any data point
    # using round because Ryan did - but won't that ruin small numbers??? TODO this
    max1 = max(y_train_true.max(), y_train_pred.max(),
               y_test_true.max(), y_test_pred.max())
    min1 = min(y_train_true.min(), y_train_pred.min(),
               y_test_true.min(), y_test_pred.min())
    max1 = round(float(max1), rounder(max1-min1))
    min1 = round(float(min1), rounder(max1-min1))
    for y_true, y_pred, stats, groups, title_addon in \
            (train_quad+('train',), test_quad+('test',)):

        # make fig and ax, use x_align when placing text so things don't overlap
        x_align=0.64
        fig, ax = make_fig_ax(x_align=x_align)

        # set tick labels
        # notice that we use the same max and min for all three. Don't
        # calculate those inside the loop, because all the should be on the same scale and axis
        _set_tick_labels(ax, max1, min1)

        # plot diagonal line
        ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

        # do the actual plotting
        if groups is None:
            ax.scatter(y_true, y_pred, color='blue', edgecolors='black', s=100, zorder=2, alpha=0.7)
        else:
            handles = dict()
            unique_groups = np.unique(np.concatenate((train_groups, test_groups), axis=0))
            log.debug(' '*12 + 'unique groups: ' +str(list(unique_groups)))
            colors = ['blue', 'red', 'green', 'purple', 'orange', 'black']
            markers = ['o', 'v', '^', 's', 'p', 'h', 'D', '*', 'X', '<', '>', 'P']
            colorcount = markercount = 0
            for groupcount, group in enumerate(unique_groups):
                mask = groups == group
                log.debug(' '*12 + f'{group} group_percent = {np.count_nonzero(mask) / len(groups)}')
                handles[group] = ax.scatter(y_true[mask], y_pred[mask], label=group, color=colors[colorcount],
                                            marker=markers[markercount], s=100, alpha=0.7)
                colorcount += 1
                if colorcount % len(colors) == 0:
                    markercount += 1
                    colorcount = 0
            ax.legend(handles.values(), handles.keys(), loc='lower right', fontsize=12)

        # set axis labels
        ax.set_xlabel('True '+label, fontsize=16)
        ax.set_ylabel('Predicted '+label, fontsize=16)

        plot_stats(fig, stats, x_align=x_align, y_align=0.90)

        filename = 'predicted_vs_true_'+ title_addon + '.png'
        filenames.append(filename)
        fig.savefig(join(outdir, filename), dpi=DPI, bbox_inches='tight')

    return filenames


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
train_quad = (array([177.28302038, 202.0924573 , 325.49788774, ...,  47.20790294,
        23.809066  ,   0.        ]), array([ 88.64152247, 101.04624221, 162.74894387, ...,  23.60395147,
        11.904533  ,   0.        ]), OrderedDict([('R2', 0.9954059447356449), ('root_mean_squared_error', 70.84772083357406), ('mean_absolute_error', 52.61313682325343), ('rmse_over_stdev', 0.7443069870284538)]), None)
test_quad = (array([1.50371026e+02, 2.22927321e+02, 2.29790128e+02, 2.41537703e+02,
       4.07147186e+02, 3.46912901e+02, 2.44558934e+02, 7.98843255e+01,
       9.96717956e+01, 1.20156284e+02, 1.06566685e+02, 1.86835775e+02,
       7.75902143e+01, 1.78590946e+02, 1.02665208e+02, 1.27306711e+02,
       1.47491884e+02, 1.01679243e+02, 1.70112526e+02, 1.71679718e+02,
       1.44367703e+02, 1.90002071e+02, 1.90460889e+02, 1.25776102e+02,
       1.72040539e+02, 1.85728826e+02, 1.37488659e+02, 7.23673571e+01,
       1.74631224e+02, 3.79332462e+01, 9.24493164e+01, 0.00000000e+00,
       9.78662706e+01, 9.51442246e+01, 2.36177315e+02, 1.08666855e+02,
       7.09734216e+01, 2.91384616e+02, 6.60766101e+01, 2.36664682e+02,
       4.08488586e+02, 5.15805330e+02, 1.22389663e+02, 8.81149314e+01,
       1.57715585e+02, 2.37521275e+02, 1.28364345e+02, 2.38334229e+02,
       3.47383100e+02, 4.16012879e+01, 9.79676617e+00, 7.48766194e+00,
       0.00000000e+00, 1.56215250e+01, 6.31512971e+01, 0.00000000e+00,
       5.64799569e+01, 1.49611167e+02, 0.00000000e+00, 0.00000000e+00,
       3.11290486e+01, 7.93342932e+01, 5.04901919e+01, 0.00000000e+00,
       2.12302521e+02, 1.98106664e+02, 1.18302658e+02, 3.77502009e+02,
       1.19774983e+02, 5.00254523e+01, 0.00000000e+00, 3.31384554e+01,
       4.09146349e+01, 3.93315709e+01, 4.89115430e+01, 0.00000000e+00,
       0.00000000e+00, 4.51430137e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.69769380e+02, 1.08224668e+02, 3.34361123e+01,
       4.06187327e+01, 6.70851015e+01, 4.33637207e+01, 3.81461657e+01,
       3.21439514e+01, 3.92412752e+01, 2.10808929e+02, 2.32209209e+02,
       1.57374363e+02, 0.00000000e+00, 1.55792869e+02, 9.57636772e+01,
       0.00000000e+00, 0.00000000e+00, 6.92693877e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.06802766e+01,
       5.53782777e+01, 4.07973166e+01, 5.08263675e+01, 1.41470544e+02,
       2.39414173e+01, 7.92325765e+01, 6.53557069e+01, 3.98312067e+01,
       2.18803383e+01, 4.48563810e+01, 1.33902387e+01, 3.61795100e+01,
       1.44194572e+02, 1.07975067e+02, 1.12573845e+02, 9.85807552e+01,
       0.00000000e+00, 2.83906453e+02, 2.02289181e-01, 1.51553671e+02,
       5.24822446e+01, 8.98167378e+01, 8.77843631e+01, 9.36717096e+01,
       0.00000000e+00, 8.69178900e+00, 4.36717688e+00, 1.27378189e+02,
       1.71956703e+02, 2.58665310e+02, 0.00000000e+00, 4.21689745e+01,
       3.15202492e+01, 1.26777344e+02, 2.29064683e+02, 1.99684569e+02,
       0.00000000e+00, 1.38924377e+01, 5.14231024e+01, 0.00000000e+00,
       0.00000000e+00, 7.81817223e+01, 4.55261914e+01, 4.79204659e+01,
       0.00000000e+00, 5.09486685e+01, 6.98113911e+01, 1.55465078e+02,
       1.32773863e+02, 2.04099993e+02, 6.77439450e+01, 7.83502295e+01,
       7.67577252e+01, 6.94770317e+01, 4.42032278e+01, 3.74738040e+01,
       5.46661575e+01, 5.67330262e+01, 0.00000000e+00, 0.00000000e+00,
       3.95429854e+01, 2.82496436e+01, 0.00000000e+00, 3.42510607e+01,
       1.62892176e+02, 1.50562981e+02, 0.00000000e+00, 1.41471166e+02,
       1.95442993e+02, 2.85835223e+01, 1.15904985e+01, 5.50100622e+01,
       7.11848257e+01, 1.03266329e+02, 6.89092871e+01, 0.00000000e+00,
       3.69821609e+00, 5.31766390e-01, 1.12780820e+01, 0.00000000e+00,
       0.00000000e+00, 4.45467226e+01, 7.37535702e+01, 0.00000000e+00,
       2.44348237e+02, 1.36431444e+02, 1.85197855e+02, 2.02269808e+02,
       2.65270592e+02, 1.32871922e+02, 3.78453371e+02, 8.83272115e+01,
       9.03169483e+01, 9.77903794e+01, 2.41664953e+02, 1.81295285e+02,
       2.49724729e+02, 2.30118336e+02, 1.14407716e+02, 1.31761462e+02,
       1.89952208e+02, 1.24477908e+02, 1.26700822e+02, 1.49948333e+02,
       1.55694637e+02, 1.50672468e+02, 1.63799175e+02, 2.18442616e+02,
       9.81125625e+01, 1.52280199e+02, 1.91836674e+02, 2.12302508e+02,
       1.78586168e+02, 1.01768108e+02, 1.56136057e+02, 1.35390845e+02,
       1.69573331e+02, 2.73613179e+02, 2.37992715e+02, 3.09856621e+02,
       2.04046571e+02, 2.55478983e+02, 3.49453381e+02, 3.18399908e+02,
       7.64960376e+01, 2.05564030e+02, 7.04378967e+01, 8.23163755e+01,
       9.77091114e+01, 8.35495499e+01, 2.08659157e+02, 1.77317832e+02,
       2.80854292e+02, 2.06610134e+02, 1.54641811e+02, 1.00117427e+02,
       1.40265182e+02, 9.48376021e+01, 9.54547321e+01, 1.83449190e+02,
       2.13333228e+02, 1.08425980e+02, 1.59401405e+02, 1.85807040e+02,
       2.40576558e+02, 2.59216571e+02, 3.15514875e+02, 1.44889042e+02,
       2.38988178e+02, 2.52611496e+02, 2.18137285e+02, 3.05414508e+02,
       7.10448978e+01, 7.89063338e+01, 6.56442278e+01, 1.70005981e+02,
       1.38575932e+02, 1.79659313e+02, 1.18314107e+02, 1.23035360e+02,
       1.36443088e+02, 5.95241562e+01, 1.29155595e+02, 5.68526124e+01,
       1.65456460e+02, 3.12115625e+02, 2.42965465e+02, 1.91404536e+02,
       2.00760493e+02, 9.56831956e+02, 1.28639131e+02, 1.23367070e+02,
       1.38844996e+02, 5.74165434e+01, 1.04682468e+02, 1.10065846e+02,
       1.43722366e+02, 1.09913097e+02, 1.94691060e+02, 1.15142225e+02,
       1.51089123e+02, 9.45208382e+01, 1.26955312e+02, 1.78159873e+02,
       1.56518089e+02, 9.04495542e+01, 6.89800768e+01, 2.66189733e+01,
       5.18182329e+01, 3.70668895e+01, 1.24273117e+02, 2.42317220e+01,
       3.26473248e+01, 7.34595746e+01, 7.19683460e+01, 4.18854297e+01,
       4.30159281e+01, 7.21745594e+01, 4.85930390e+01, 9.42635786e+01,
       4.29043523e+01, 1.73730028e+02, 1.76433607e+02, 2.10616630e+02,
       1.60456727e+02, 1.67324805e+02, 2.11267960e+02, 1.87207178e+02,
       2.36861914e+02, 1.33312148e+02, 1.54672899e+02, 2.00391998e+02,
       9.14620807e+01, 1.00815664e+02, 2.71319339e+02, 1.37479653e+02,
       1.10285836e+02, 1.35954460e+02, 1.83271284e+02, 1.17247059e+02,
       1.37791901e+02, 1.56027039e+02, 1.35955483e+02, 2.16122195e+02,
       1.36808521e+02, 1.35369317e+02, 1.21692419e+02, 9.37481720e+01,
       7.98648319e+01, 1.34354983e+02, 3.33198729e+01, 2.83093128e+02,
       2.23792303e+01, 1.09582964e+02, 9.88464407e+01, 1.40488835e+02,
       1.11369551e+02, 1.92009283e+02, 1.04314376e+01, 9.43179777e+01,
       1.32019981e+02, 1.78944559e+02, 9.04350450e+01, 0.00000000e+00,
       1.55909864e+02, 9.38423029e+01, 7.18885960e+01, 9.37906658e+01,
       5.63388158e+01, 4.54043655e+01, 6.24481656e+01, 0.00000000e+00,
       1.01755733e+02, 2.83147240e+01, 2.55044144e+00, 6.11195316e+01,
       2.86637589e+01, 7.99376675e+00, 0.00000000e+00, 3.23684503e+01,
       1.20026452e+02, 3.86517633e+01, 2.77992144e+01, 1.72121127e+01,
       0.00000000e+00, 0.00000000e+00, 1.84387405e+02, 7.65032664e+01,
       6.74148765e+01, 6.23324679e+01, 3.60966810e+01, 9.16707430e+01,
       1.16752352e+02, 4.08206874e+01, 4.60103452e+01, 3.88036349e+01,
       8.61157898e+01, 8.42613186e+01, 8.65192126e+01, 5.33006835e+01,
       6.76609259e+01, 1.05624926e+02, 4.11379073e+01, 2.72516127e+01,
       4.63580983e+01, 3.15042600e+01, 3.05960388e+01, 4.59371875e+01,
       2.93690973e+01, 9.40057842e+01, 4.67667976e+01, 1.20061041e+02,
       2.84167104e+01, 4.54076116e+01, 4.53272821e+01, 5.00275181e+01,
       8.97135102e+01, 1.25245007e+02, 7.00096850e+00, 7.80670028e+01,
       7.76829500e+01, 1.34709235e+01, 5.97432500e+00, 7.83039861e+01,
       2.19120080e+01, 7.17549475e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 8.66887169e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00]), array([ 2.37162407e-005,  0.00000000e+000,  1.57670412e-122,
        0.00000000e+000,  3.21633711e-005,  5.24841590e-236,
        0.00000000e+000,  4.54344107e-028,  2.06042835e-231,
        5.60757180e-192,  0.00000000e+000,  8.99141021e-133,
        1.98940071e-027,  3.94752770e-027,  3.39871916e-231,
        1.05056155e-132,  4.59397478e-192,  0.00000000e+000,
        7.51332792e-192,  0.00000000e+000,  1.93017472e-010,
        2.25516103e-130,  8.03909803e-236,  0.00000000e+000,
        0.00000000e+000,  5.69420679e-032,  1.07967237e-132,
        0.00000000e+000,  1.05517298e-252,  7.66926972e-258,
        6.18256310e-013,  1.53153616e-157,  3.41500373e-001,
        5.12641420e-027,  9.92729010e-111,  1.59368346e-103,
        0.00000000e+000,  1.74801913e-115,  1.61738315e-020,
        7.14212545e-004,  3.48980138e-044,  8.33153752e-125,
        1.00100267e-090,  1.49372738e-022,  1.84682607e-092,
        4.20249123e-002,  9.65766292e-006,  1.17461841e-025,
        5.39575881e-040,  9.35854752e-099,  0.00000000e+000,
        1.29774479e-106,  3.23319374e-108,  0.00000000e+000,
        9.59903241e-027,  8.25336855e-011,  4.42365260e-003,
        1.04022706e-020,  0.00000000e+000,  4.30929508e-180,
        3.50452414e-067,  3.53193397e-029,  1.46674629e-023,
        2.08741624e-054,  0.00000000e+000,  9.51277628e-045,
        1.37469631e-005,  0.00000000e+000,  1.58884929e-001,
        5.72914762e-095,  0.00000000e+000,  3.32575071e-072,
        1.43238441e-012,  5.32517745e-006,  9.95516546e-002,
        8.99453929e-239,  5.74151822e-241,  4.73036284e-238,
        0.00000000e+000,  0.00000000e+000,  6.58523341e-031,
       -1.48090289e-105, -4.94570825e-076,  4.52271122e-097,
        2.69747241e-235,  7.70760338e-239, -1.28914187e-007,
        0.00000000e+000,  5.59586356e-002,  3.90799562e-084,
        1.72505857e-033,  5.99006251e-008,  1.22193215e-012,
        9.64462759e-117,  3.19095140e-084,  2.49239196e-095,
        2.03563057e-303,  2.17502467e-239,  6.70779449e-084,
        1.35637112e-094,  6.86576022e-010,  0.00000000e+000,
        1.43118692e-071,  7.11505024e-005,  0.00000000e+000,
        0.00000000e+000,  2.23444414e+000, -2.14724096e-146,
        1.34949156e-241,  5.75700245e-240,  7.91172700e-240,
        2.09913303e-242,  2.27881472e-242, -3.94692655e-088,
        6.83274234e-096,  1.98846757e-010,  4.30310055e-012,
        6.10181038e-001,  2.41910045e-006,  4.31746609e-009,
        3.19772012e-089,  2.41692700e-004,  1.99731957e-003,
        1.44580073e-030,  2.53105552e-095,  1.71695051e-010,
        1.49563477e-006,  5.50334199e-003,  2.40953996e-161,
        0.00000000e+000,  1.04351398e-162,  0.00000000e+000,
        5.76416295e-007,  3.04575136e-005,  4.41746395e-094,
        6.01608154e-049,  3.46064758e-006,  4.49083689e+001,
        7.78285888e-010,  9.07968332e-010,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  7.45936682e-023,
        3.15327144e-213,  1.69322763e-048,  1.14122705e-056,
        5.58005897e-156,  3.10207527e-098,  6.18648971e-010,
        4.27243789e-011,  1.33494490e-023,  0.00000000e+000,
        0.00000000e+000,  1.77543543e-240, -1.13506940e-027,
        0.00000000e+000,  9.81655039e-088,  3.04281346e-011,
        1.13014587e-004,  0.00000000e+000,  4.89465128e+000,
        7.94957609e-004,  1.83111272e-087,  0.00000000e+000,
        0.00000000e+000,  1.42764085e-237,  7.37555700e-239,
        1.18108619e-007,  1.39166377e-003,  6.18462520e-006,
        2.77400168e-026,  4.56234169e-007,  1.73944097e-006,
        0.00000000e+000,  0.00000000e+000,  9.99740503e+000,
        3.46459644e-077,  8.13512929e-009,  5.67010675e-003,
        1.67141215e-087,  4.75936101e-006,  0.00000000e+000,
        1.99618584e-007,  1.55868612e-076,  4.49020499e-004,
        7.91391488e-089,  3.16055502e-002,  9.06351315e-133,
        5.17850042e-133,  1.68368267e-003,  0.00000000e+000,
        5.84639034e+000,  4.55445253e-078,  7.71465973e-003,
        2.79718708e-199,  0.00000000e+000,  9.13582395e-062,
        6.64782738e-027,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  4.05099155e+000,  6.26723852e+000,
        7.35982868e-011,  3.45645532e-001,  7.67550218e-001,
        6.10556997e-032,  4.09243806e-020,  2.11922523e-001,
        5.36558010e-001,  2.21316506e-005,  0.00000000e+000,
        0.00000000e+000,  2.77804681e+000,  3.15884301e+000,
        3.65249042e-001,  2.36213365e-001,  2.50888735e-001,
        4.68118952e-001,  3.62198440e-001,  2.62778285e-001,
        2.94240547e-001,  5.62908337e-002,  2.37057224e-001,
        2.77050197e-001,  3.77843055e-001,  2.06843597e+001,
        1.91693684e+001,  1.07130365e+001,  3.95419122e+000,
        2.70103344e+000,  4.45970404e-004,  5.67465023e-001,
        6.45481106e+000,  9.34004877e+000,  6.37583699e+000,
        5.20155306e+000,  7.50160537e+000,  7.57432041e+000,
        1.86746244e+000,  4.55897313e+000,  2.12051267e-001,
        4.03892147e-001,  4.41927862e+001,  3.21279779e-001,
        3.52821994e-001,  4.47896744e-002,  4.07735967e-008,
        3.82071506e-001,  2.73069006e-001,  1.86485140e-001,
        3.19385646e-008,  9.52423662e+000,  1.01629204e+001,
        1.90129753e+001,  5.22669044e+000,  2.69381775e+000,
        3.53507004e-047,  2.61216023e-032,  1.10492183e-010,
        5.21126744e-001,  6.98451758e-001,  5.49006469e-008,
        7.45997558e+000,  4.96785934e+000,  5.50594151e+000,
        7.20007614e+000,  1.22370299e-019,  5.74662722e-014,
        1.53848184e-001,  1.70266534e-001,  3.16565244e-008,
        4.52935249e-001,  1.90057960e-001,  2.32190384e-001,
        3.28993927e-002,  1.15986629e-001,  1.33537102e+001,
        6.20996534e+000,  4.11126557e+000,  6.89767697e-001,
        6.57950206e+000,  5.01138620e+000,  5.29482942e+000,
        3.95179893e-016,  3.03867444e-006,  2.75363190e-071,
        1.53713441e-028,  2.42981258e-017,  4.62685175e-104,
        5.24851123e-026,  1.01526317e-003,  9.77096931e-004,
        1.47282066e-003,  2.54490443e-001,  3.37088062e-008,
        5.79671542e-017,  5.90852643e-017,  1.47742493e-003,
        3.16273295e-008,  1.51707962e-006,  5.28002773e-011,
        1.94316774e-001,  4.48372794e-011,  5.75407616e-006,
        6.29313121e-006,  1.07191082e-156,  3.22518280e-069,
        6.45953608e-027,  5.89661082e-004,  3.81338889e-004,
        3.49139298e-001,  2.28307373e-003,  3.37731994e-004,
        5.18744411e-006,  2.29679230e-004,  4.90144024e-015,
        7.89232543e-012,  5.70760500e-016,  5.84862720e-025,
        1.47594322e-158,  1.69555021e-003,  1.21936520e-002,
        1.71217402e-003,  1.14620490e-003,  1.33365812e-003,
        9.00654637e-012,  2.63345382e+000,  1.93223286e-001,
        0.00000000e+000,  3.31022099e+000,  8.49186459e-004,
        6.92560973e-001,  2.32457660e+000,  2.84534534e+000,
        8.85514140e-004,  2.46328676e-001,  4.68466563e+000,
        1.10358723e-003,  9.40980826e-005,  1.38550075e+001,
        9.89772228e-002,  2.40106187e-004,  8.41737525e+000,
        3.10974832e+000,  9.88273420e-026,  1.22729862e-012,
        4.93742780e-020,  1.03814829e-002,  1.46798274e-012,
        8.46374012e-062,  1.06733079e-085,  2.98606309e-037,
        0.00000000e+000,  0.00000000e+000,  1.18588239e-021,
        4.47663279e-105,  2.85767271e-098,  0.00000000e+000,
        1.30341203e-035,  9.26888140e-018,  5.88972980e-005,
        5.16865374e-078,  6.50299074e-028,  7.73577970e-011,
        2.57603703e-001,  0.00000000e+000,  4.10348027e-080,
        2.88924640e-031,  2.45638971e-018,  5.78659855e-063,
        7.58887726e-002,  2.05524312e-013,  0.00000000e+000,
        4.26469187e-003,  2.16092991e-080,  4.53656851e-011,
        7.24940183e-001,  2.53378165e-003,  0.00000000e+000,
        0.00000000e+000,  6.83244198e-080,  1.12669156e-030,
        1.15796258e-028,  8.94566928e-006,  3.36024419e-026,
        7.49154520e-076,  4.76574120e-033,  7.94777692e-003,
        3.75733131e-013,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  1.18312935e-030,  1.50576689e-001,
        4.46962629e-005,  3.19017750e+000,  4.60703029e-028,
        2.67472686e-019,  5.08243692e-031,  4.39222912e-011,
        8.24054618e-018,  2.61836652e-017,  4.79836376e-009,
        5.48898771e-008,  1.27623698e-069,  1.62616009e-103,
        0.00000000e+000,  6.85751162e-028,  0.00000000e+000,
        0.00000000e+000,  2.95994854e-256,  3.21720601e-263,
        4.70350495e-321,  3.23202800e-006,  2.01906192e-004,
        1.24511624e-020,  1.30644998e-052,  1.37554542e-003,
        1.16496389e-030,  8.41756743e-081,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  2.26689970e-004,
        0.00000000e+000,  0.00000000e+000,  2.51722038e-006]), OrderedDict([('R2', 0.014196104575853852), ('root_mean_squared_error', 142.62145741500382), ('mean_absolute_error', 105.60544726215772), ('rmse_over_stdev', 1.4983424449677283)]), None)
outdir = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge/RepeatedKFold/split_0'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_paths = plot_predicted_vs_true(train_triple, test_triple, outdir)
for plot_path in plot_paths:
    display(Image(filename=plot_path))
